In [1]:
import polars as pl
import plotly.express as px
import pyarrow as pa

In [2]:
students = pl.read_csv("../data/students.csv")
grades = pl.read_csv("../data/final_table.csv")
grades = grades.with_columns(
    pl.col('Sigla').str.slice(0, 3)
    .alias('Departament'),
    pl.col('student_id').alias('IdEstudante')
    )
grades_with_students = grades.join(students, on='IdEstudante', how='left')
grades_with_students

Sigla,Nome,Ano/Semestre,AU,TR,AU+TR,EXT,FREQ,NOTA,student_id,Departament,IdEstudante,Idade,Curso,AnoIngresso,Ponderada,Classificacao,AnoConclusao,AAC,AEX
str,str,str,i64,i64,i64,i64,f64,f64,i64,str,i64,i64,str,i64,f64,i64,i64,i64,i64
"""SCC0200""","""Informação Profissional e Tuto…","""2023/1""",2,0,2,30,100.0,10.0,0,"""SCC""",0,20,"""BCC""",2023,9.1,16,2027,80,190
"""SCC0221""","""Introdução à Ciência de Comput…","""2023/1""",4,1,5,90,97.0,9.0,0,"""SCC""",0,20,"""BCC""",2023,9.1,16,2027,80,190
"""SCC0222""","""Laboratório de Introdução à Ci…","""2023/1""",2,2,4,90,100.0,9.8,0,"""SCC""",0,20,"""BCC""",2023,9.1,16,2027,80,190
"""SMA0300""","""Geometria Analítica""","""2023/1""",4,0,4,60,100.0,7.5,0,"""SMA""",0,20,"""BCC""",2023,9.1,16,2027,80,190
"""SMA0353""","""Cálculo I""","""2023/1""",4,0,4,60,100.0,10.0,0,"""SMA""",0,20,"""BCC""",2023,9.1,16,2027,80,190
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SME0104""","""Cálculo Numérico""","""2025/1""",4,0,4,0,100.0,9.5,4,"""SME""",4,20,"""BCC""",2023,8.9,32,2027,1,180
"""SME0121""","""Processos Estocásticos""","""2025/1""",4,0,4,0,80.0,8.5,4,"""SME""",4,20,"""BCC""",2023,8.9,32,2027,1,180
"""SME0130""","""Redes Complexas""","""2025/1""",4,2,6,0,100.0,10.0,4,"""SME""",4,20,"""BCC""",2023,8.9,32,2027,1,180


In [4]:
# plot grade average per department

dept_avg = grades_with_students.group_by('Departament').agg(pl.col('NOTA').mean())
plot = px.bar(dept_avg.to_pandas(), x='Departament', y='NOTA', title='Average Grade per Department', color='Departament')
plot.show()

In [11]:
# Grades per semester

grades_with_rel_semester = grades_with_students.with_columns(
    pl.col('Ano/Semestre').str.split('/').list.get(0).cast(pl.Int32).alias('Ano'),
    pl.col('Ano/Semestre').str.split('/').list.get(1).cast(pl.Int32).alias('Semestre')
    ).with_columns(
        ((pl.col('Ano')-pl.col('AnoIngresso'))*2 + pl.col('Semestre')).alias('RelSemester')
        )

grades_with_rel_semester


Sigla,Nome,Ano/Semestre,AU,TR,AU+TR,EXT,FREQ,NOTA,student_id,Departament,IdEstudante,Idade,Curso,AnoIngresso,Ponderada,Classificacao,AnoConclusao,AAC,AEX,Ano,Semestre,RelSemester
str,str,str,i64,i64,i64,i64,f64,f64,i64,str,i64,i64,str,i64,f64,i64,i64,i64,i64,i32,i32,i64
"""SCC0200""","""Informação Profissional e Tuto…","""2023/1""",2,0,2,30,100.0,10.0,0,"""SCC""",0,20,"""BCC""",2023,9.1,16,2027,80,190,2023,1,1
"""SCC0221""","""Introdução à Ciência de Comput…","""2023/1""",4,1,5,90,97.0,9.0,0,"""SCC""",0,20,"""BCC""",2023,9.1,16,2027,80,190,2023,1,1
"""SCC0222""","""Laboratório de Introdução à Ci…","""2023/1""",2,2,4,90,100.0,9.8,0,"""SCC""",0,20,"""BCC""",2023,9.1,16,2027,80,190,2023,1,1
"""SMA0300""","""Geometria Analítica""","""2023/1""",4,0,4,60,100.0,7.5,0,"""SMA""",0,20,"""BCC""",2023,9.1,16,2027,80,190,2023,1,1
"""SMA0353""","""Cálculo I""","""2023/1""",4,0,4,60,100.0,10.0,0,"""SMA""",0,20,"""BCC""",2023,9.1,16,2027,80,190,2023,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SME0104""","""Cálculo Numérico""","""2025/1""",4,0,4,0,100.0,9.5,4,"""SME""",4,20,"""BCC""",2023,8.9,32,2027,1,180,2025,1,5
"""SME0121""","""Processos Estocásticos""","""2025/1""",4,0,4,0,80.0,8.5,4,"""SME""",4,20,"""BCC""",2023,8.9,32,2027,1,180,2025,1,5
"""SME0130""","""Redes Complexas""","""2025/1""",4,2,6,0,100.0,10.0,4,"""SME""",4,20,"""BCC""",2023,8.9,32,2027,1,180,2025,1,5


In [20]:
# Plot grade average per relative semester with plotly

average_grades_per_semester = grades_with_rel_semester.group_by('RelSemester').agg(
    pl.col('NOTA').mean().alias('AverageGrade')
    ).sort('RelSemester')

fig = px.line(average_grades_per_semester.filter(pl.col('RelSemester') < 6), x='RelSemester', y='AverageGrade', title='Média das Notas por Semestre Relativo')
fig.update_xaxes(dtick=1)
fig.show()

In [29]:
# sum AU per semester with plotly

# plot average AU sum per student per relative semester
au_per_student_semester = grades_with_rel_semester.group_by(['IdEstudante', 'RelSemester']).agg(
    pl.col('AU').sum().alias('TotalAU')
    )

au_per_student_semester.sort(['IdEstudante', 'RelSemester'])


IdEstudante,RelSemester,TotalAU
i64,i64,i64
0,1,24
0,2,28
0,3,19
0,4,17
0,5,24
…,…,…
4,1,24
4,2,28
4,3,21


In [30]:
average_au_per_semester = au_per_student_semester.group_by('RelSemester').agg(
    pl.col('TotalAU').mean().alias('AverageAU')
    ).sort('RelSemester')
fig2 = px.line(average_au_per_semester.filter(pl.col('RelSemester') < 6), x='RelSemester', y='AverageAU', title='Média de AU por Semestre Relativo')
fig2.update_xaxes(dtick=1)
fig2.show()


In [40]:
# plot names per frequency average, to identify lesson names with low frequency
name_frequency = grades_with_rel_semester.filter(pl.col('RelSemester') < 6).group_by('Nome').agg(
    pl.col('FREQ').mean().alias('AverageFrequency'))
name_frequency_sorted = name_frequency.sort('AverageFrequency')
name_frequency_sorted

Nome,AverageFrequency
str,f64
"""Cálculo IV""",70.0
"""Redes de Computadores""",74.0
"""Introdução ao Desenvolvimento …",74.8
"""Prática em Organização de Comp…",76.0
"""Análise e Projeto Orientados a…",76.6
…,…
"""Laboratório de Introdução à Ci…",100.0
"""Tópicos Avançados em Ciências …",100.0
"""Cálculo I""",100.0


In [ ]:
fig3 = px.bar(name_frequency_sorted.head(10).to_pandas(), x='AverageFrequency', y='Nome', title='Top 10 Disciplinas com Menor Frequência', color='AverageFrequency', orientation='h')
fig3.show()

In [47]:
# Grades scatter plot per AU+TR

fig4 = px.scatter(grades_with_rel_semester.filter(pl.col('RelSemester') < 6).to_pandas(), x='AU+TR', y='NOTA', title='Dispersão de Notas por AU e TR')
fig4.show()

In [50]:
# Count departament occurrences per student
dept_counts = grades_with_students.group_by(['IdEstudante', 'Departament']).agg(
    pl.len().alias('Count')
    ).sort(['IdEstudante', 'Count'])
dept_counts

IdEstudante,Departament,Count
i64,str,u32
0,"""760""",2
0,"""SME""",4
0,"""SMA""",5
0,"""SCC""",12
0,"""SSC""",13
…,…,…
4,"""760""",2
4,"""SME""",5
4,"""SMA""",6


In [57]:
# Plot departament count per student
fig5 = px.bar(dept_counts.to_pandas(), x='IdEstudante', y='Count', color='Departament', title='Contagem de Departamentos por Estudante', barmode='group', orientation='v')
fig5.show()

In [ ]:
# Sum EXT per student
ext_per_student = grades_with_students.group_by('IdEstudante').agg(
    pl.col('EXT').sum().alias('TotalEXT')
    ).sort('TotalEXT')
ext_per_student